In [ ]:
import pandas as pd

# Load Good units
df_loaded = pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/functional_properties_with_python_measurements_pycirc.pkl")
df_loaded 



In [ ]:
def get_base_name(col):
    # 去掉 _x, _y, _z 等后缀
    return col.rsplit('_', 1)[0] if '_' in col and col.endswith(('_x', '_y', '_z')) else col

# 获取列名分组
base_names = {}
for col in df_loaded.columns:
    base = get_base_name(col)
    if base not in base_names:
        base_names[base] = []
    base_names[base].append(col)

# 2. 只保留每个基名的第一个列，并重命名去掉后缀
keep_columns = []
for base, cols in base_names.items():
    if len(cols) > 1:
        # 如果有重复列，保留第一个，去掉后缀
        keep_columns.append((cols[0], base))
    else:
        # 没有后缀的列直接保留
        keep_columns.append((cols[0], cols[0]))

# 3. 选择保留的列并重命名
selected_columns = [col for col, _ in keep_columns]
df_cleaned = df_loaded[selected_columns].copy()
df_cleaned.columns = [new_name for _, new_name in keep_columns]

# 4. 可选：重置索引
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned.to_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/functional_properties_with_python_measurements_pycirc.pkl")

In [ ]:
from scipy import stats
control_ks_stat, control_ks_p = stats.shapiro(control_values)
control_ks_p

In [ ]:
plt.hist(control_df["theta_index"],bins=50)

In [ ]:
plt.hist(exp_df["theta_index"],bins=50)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Load Good units
#df_loaded = pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl")
df_good = df_cleaned[df_cleaned['unit_quality'] == "good"]
df_py = df_good[(df_good['buzaki_py_cell_type'] == "pyramidal")]

#df_phase_precessing =  df_py[(df_py['PS_p1'] < 0.05) & (df_py['theta_index'] > 5)]  
df_phase_precessing =  df_py
funct = [""]

for functional_plot in funct:
    df = df_phase_precessing

    base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/cell_type"

    control_ids = ['65165', '65091', '63383', '66539', '65622']
    exp_ids = ['65588', '63385', '66538', '66537', '66922']

    session = ["A"]

    for session in session:
        if session == "Total":
            df_a = df
        else:
            df_a = df[df['session'] == session]

        control_df = df_a[df_a['animal_id'].isin(control_ids)]
        exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

        sns.set_theme(style="ticks")

        metrics = ['PS_slope1', 'PS_slope2', "theta_index",'PS_cr1','PS_cr2','PS_cr3']

        fig_violin, axes_violin = plt.subplots(2, 3, figsize=(14, 10))
        axes_violin = axes_violin.flatten()

        fig_cdf, axes_cdf = plt.subplots(2, 3, figsize=(14, 10))
        axes_cdf = axes_cdf.flatten()

        control_color = sns.color_palette(palette='flag')[-1]
        exp_color = "cyan"

        for idx, metric in enumerate(metrics):
            control_values = control_df[metric].dropna().reset_index(drop=True)
            exp_values = exp_df[metric].dropna().reset_index(drop=True)


            if len(control_values) > 0 and len(exp_values) > 0:
                control_mean = control_values.mean()
                exp_mean = exp_values.mean()
                control_sem = control_values.sem()
                exp_sem = exp_values.sem()

                print(f"\nComparison for {metric} (Session {session}):")
                print(f"Control mean: {control_mean:.2f} ± {control_sem:.2f}")
                print(f"Experimental mean: {exp_mean:.2f} ± {exp_sem:.2f}")

                # Statistical tests for violin plot (one-sided, control > exp)
                control_ks_stat, control_ks_p = stats.shapiro(control_values)
                # control_ks_stat, control_ks_p = stats.kstest(control_values, 'norm', 
                #                                             args=(control_mean, control_values.std()))
                
                exp_ks_stat, exp_ks_p = stats.shapiro(exp_values)
                normal = control_ks_p > 0.05 and exp_ks_p > 0.05

                levene_stat, levene_p = stats.levene(control_values, exp_values)
                homoscedastic = levene_p > 0.05

                if normal and homoscedastic:
                    t_stat, p_val = stats.ttest_ind(control_values, exp_values, equal_var=True)
                    test_name = " t-test"
                    print(f"One-sided t-test statistic: {t_stat:.2f}, p-value: {p_val:.4f}")
                else:
                    x = np.asarray(control_values.dropna(), dtype=float)
                    y = np.asarray(exp_values.dropna(), dtype=float)

                    u_stat, p_val = stats.mannwhitneyu(x, y, alternative='less')
                    test_name = " Mann-Whitney U"
                    print(f"One-sided Mann-Whitney U statistic: {u_stat:.2f}, p-value: {p_val:.4f}")

                # Two-sample KS test (one-sided, control > exp)
                ks_stat, ks_p = stats.ks_2samp(control_values, exp_values, alternative='less')
                print(f" KS statistic: {ks_stat:.2f}, p-value: {ks_p:.4f}")

                # Two-sample KS test for CDF
                ks_stat, ks_p = stats.ks_2samp(control_values, exp_values)
                print(f"KS statistic: {ks_stat:.2f}, p-value: {ks_p:.4f}")

                n_con = len(control_values)
                n_exp = len(exp_values)
                print(fr"session {session}, n con {n_con}, n exp {n_exp}")
                # Violin Plot
                plot_df = pd.DataFrame({
                    'value': pd.concat([control_values, exp_values]),
                    'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
                })
                all_values = plot_df['value']
                mean_val = all_values.mean()
                std_val = all_values.std()
                plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                           (plot_df['value'] <= mean_val + 3 * std_val)]

                sns.violinplot(
                    data=plot_df_filtered, x='group', y='value', ax=axes_violin[idx],
                    palette={"Control": control_color, "Experimental": exp_color}, width=0.8, cut=0
                )
                for patch in axes_violin[idx].collections:
                    patch.set_alpha(1)

                sns.stripplot(
                    data=plot_df_filtered, x='group', y='value', ax=axes_violin[idx], size=4,
                    hue='group', palette={"Control": "black", "Experimental": "black"},
                    alpha=0.4, jitter=0.3, legend=False
                )

                axes_violin[idx].set_title(f'{metric} Comparison')
                axes_violin[idx].set_ylabel(metric)
                axes_violin[idx].set_xlabel('')
                axes_violin[idx].yaxis.grid(False)
                axes_violin[idx].text(0.5, 0.04, f'KS p_ctrl = {control_ks_p:.4f}, p_exp = {exp_ks_p:.4f}', 
                                     ha='center', va='top', transform=axes_violin[idx].transAxes, fontsize=8)
                axes_violin[idx].text(0.5, 1.002, f'{test_name} p = {p_val:.4f}', 
                                     ha='center', va='top', transform=axes_violin[idx].transAxes, fontsize=10)
                axes_violin[idx].spines['top'].set_visible(False)
                axes_violin[idx].spines['right'].set_visible(False)
                axes_violin[idx].spines['bottom'].set_visible(True)
                axes_violin[idx].spines['left'].set_visible(True)

                # CDF Plot
                control_cfreq = stats.cumfreq(control_values, numbins=100)
                exp_cfreq = stats.cumfreq(exp_values, numbins=100)

                control_cdf = control_cfreq.cumcount / control_cfreq.cumcount[-1]
                exp_cdf = exp_cfreq.cumcount / exp_cfreq.cumcount[-1]

                num_bins = len(control_cfreq.cumcount)
                control_x = control_cfreq.lowerlimit + np.linspace(0, control_cfreq.binsize * num_bins, num_bins)
                num_bins = len(exp_cfreq.cumcount)
                exp_x = exp_cfreq.lowerlimit + np.linspace(0, exp_cfreq.binsize * num_bins, num_bins)

                axes_cdf[idx].step(control_x, control_cdf, label='Control', color=control_color, linewidth=1.5)
                axes_cdf[idx].step(exp_x, exp_cdf, label='Experimental', color=exp_color, linewidth=1.5)


                axes_cdf[idx].set_title(f'{metric} CDF')
                axes_cdf[idx].set_xlabel(metric)
                axes_cdf[idx].set_ylabel('Cumulative Probability')
                axes_cdf[idx].legend(loc='lower right')
                axes_cdf[idx].grid(True, linestyle='--', alpha=0.7)
                axes_cdf[idx].text(0.5, 1.05, f'KS p = {ks_p:.4f}', 
                                  ha='center', va='bottom', transform=axes_cdf[idx].transAxes, fontsize=10)
                axes_cdf[idx].spines['top'].set_visible(False)
                axes_cdf[idx].spines['right'].set_visible(False)

        fig_violin.savefig(fr'{base_folder}/{session}_waveforms_pyramidal_{functional_plot}.eps', format='eps', bbox_inches='tight')
        fig_violin.savefig(fr'{base_folder}/{session}_waveforms_pyramidal_{functional_plot}.png', format='png', bbox_inches='tight')

        fig_cdf.savefig(fr'{base_folder}/{session}_waveforms_pyramidal_{functional_plot}_cdf.eps', format='eps', bbox_inches='tight')
        fig_cdf.savefig(fr'{base_folder}/{session}_waveforms_pyramidal_{functional_plot}_cdf.png', format='png', bbox_inches='tight')

        plt.tight_layout()
        plt.show()

In [ ]:
import pandas as pd

# Load Good units
df_loaded = pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper/tables/good_units_with_tsnLabels_withphase.pkl")

df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df_py = df_good[(df_good['cell_type'] == "pyramidal")]

df_phase_precessing =  df_py[(df_py['PS_p1'] < 0.05) & (df_py['PS_cr1'] < 0) &(df_good['theta_index'] > 5)]  
funct = [""]

# Define control and experimental animal IDs
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']

# Sessions to analyze
sessions = ["A", "B", "C", "Total"]

# Calculate and print ratios for each session
for session in sessions:
    if session == "Total":
        df_py_session = df_py
        df_phase_session = df_phase_precessing
    else:
        df_py_session = df_py[df_py['session'] == session]
        df_phase_session = df_phase_precessing[df_phase_precessing['session'] == session]

    # Split into control and experimental groups
    control_py = df_py_session[df_py_session['animal_id'].isin(control_ids)]
    exp_py = df_py_session[df_py_session['animal_id'].isin(exp_ids)]
    control_phase = df_phase_session[df_phase_session['animal_id'].isin(control_ids)]
    exp_phase = df_phase_session[df_phase_session['animal_id'].isin(exp_ids)]

    # Calculate ratios
    control_ratio = len(control_phase) / len(control_py) if len(control_py) > 0 else 0
    exp_ratio = len(exp_phase) / len(exp_py) if len(exp_py) > 0 else 0

    # Print results
    print(f"\nSession: {session}")
    print(f"Control - Pyramidal cells: {len(control_py)}, Phase precessing cells: {len(control_phase)}, Ratio: {control_ratio:.4f}")
    print(f"Experimental - Pyramidal cells: {len(exp_py)}, Phase precessing cells: {len(exp_phase)}, Ratio: {exp_ratio:.4f}")